<a href="https://colab.research.google.com/github/Ygrick/Data-Science/blob/main/vk_api_parsing_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Кол-во подписчиков страницы ВК](https://ru.stackoverflow.com/questions/1139039/Кол-во-подписчиков-страницы-ВК)

[habr](https://qna.habr.com/q/912927)

[Метод VK. groups.getMembers](https://dev.vk.com/method/groups.getMembers)

[Метод VK. groups.search](https://dev.vk.com/method/groups.search)

[Метод VK. groups.getAddresses](https://dev.vk.com/method/groups.getAddresses)

[Метод VK. wall.get](https://dev.vk.com/method/wall.get)

In [ ]:
!pip install vk_requests

In [ ]:
import vk_requests
import time
import pandas as pd
from datetime import datetime
import os
import urllib.request

In [ ]:
token = '57654f39ff46921683dff5189464eaca5fe78b8780a8379102d4c0de2f03bb2be46e6972761f9217fa3ae'
time_2022 = 1640984400

In [ ]:
api = vk_requests.create_api(service_token=token)

In [ ]:
# поиск 100 групп "Подслушано" с сортировкой по количеству пользователей и типом "сообщество" 
groups = api.groups.search(q='Подслушано', 
                           type='page', 
                           sort=6, 
                           count=100)

In [ ]:
groups['items']

[{'id': 34215577,
  'is_admin': 0,
  'is_advertiser': 0,
  'is_closed': 0,
  'is_member': 0,
  'name': 'Подслушано – Здесь говорят о тебе',
  'photo_100': 'https://sun1-57.userapi.com/s/v1/ig2/mfe_J90jKMVlDFfZ87DhMbYakXFp5Md2jCXbK39cjXWUfbhvuaTZO25RYp7GzaibCPgkZN61hEKq5_jmDyUPQSHw.jpg?size=100x100&quality=95&crop=0,312,1545,1545&ava=1',
  'photo_200': 'https://sun1-57.userapi.com/s/v1/ig2/K0-B_YjPiYZOFvyeQSOK5gaEZrvXT5Jf7yAVgcZWNcWKwREn8tt62U-SgBxydSnvLCjGELHqszNKuDssQCFBwbAW.jpg?size=200x200&quality=95&crop=0,312,1545,1545&ava=1',
  'photo_50': 'https://sun1-57.userapi.com/s/v1/ig2/hVBqa5hCMpGtKorlwinvi6tCt5e0zW1p0iWKu5esMEgKzymj3u2I7_3ksMgd8jDmvs7ZnIRzZPSJGeuyjw5d_7pz.jpg?size=50x50&quality=95&crop=0,312,1545,1545&ava=1',
  'screen_name': 'overhear',
  'type': 'page'},
 {'id': 76791337,
  'is_admin': 0,
  'is_advertiser': 0,
  'is_closed': 0,
  'is_member': 0,
  'name': 'Подслушано | Пошлое 18+ 16+',
  'photo_100': 'https://sun1-95.userapi.com/s/v1/if1/BI0ENSBDwWBCMyn4LaWyuCx8M6JkJwU

In [ ]:
# взятие основных полей (id, тип закрытости, имя)
list_groups = list((map(lambda item: {'id': item['id'], 
                                           'is_closed': item['is_closed'], 
                                           'name': item['screen_name'], 
                                           }, groups['items'])))

In [ ]:
# взятие количества участников (для рассчёта метрики)
for group in list_groups:
  try:
    # берём участников
    group['volume_users'] = api.groups.getMembers(group_id=group['id'], 
                                           count=0)['count']
    time.sleep(0.33)
  except:
    # удаляем группу если с группой возникает какая-либо ошибка (к примеру участники скрыты)
    list_groups.remove(group)

In [ ]:
# взятие только 50ти групп
list_groups = list_groups[:50]

In [ ]:
list_groups

[{'id': 34215577, 'is_closed': 0, 'name': 'overhear', 'volume_users': 4031170},
 {'id': 76791337,
  'is_closed': 0,
  'name': 'club76791337',
  'volume_users': 731473},
 {'id': 148881515,
  'is_closed': 0,
  'name': 'club148881515',
  'volume_users': 699584},
 {'id': 144260965, 'is_closed': 0, 'name': 'pmmgo', 'volume_users': 674921},
 {'id': 35179056,
  'is_closed': 0,
  'name': 'intimate_fact',
  'volume_users': 496836},
 {'id': 60397113,
  'is_closed': 0,
  'name': 'podslushanoperm',
  'volume_users': 479792},
 {'id': 160966228,
  'is_closed': 0,
  'name': 'club160966228',
  'volume_users': 476585},
 {'id': 84808204,
  'is_closed': 0,
  'name': 'overhear_women_secrets',
  'volume_users': 392308},
 {'id': 165972901, 'is_closed': 0, 'name': 'casesmen', 'volume_users': 377635},
 {'id': 42949290,
  'is_closed': 0,
  'name': 'overstory163',
  'volume_users': 365591},
 {'id': 80142104,
  'is_closed': 0,
  'name': 'overhear_in_football',
  'volume_users': 344005},
 {'id': 159631764,
  'is_

In [ ]:
# функция взятия только нужных значений (id группы, id поста, картинки поста если есть)
def set_posts(item):
  owner_id = item['owner_id']
  id = item['id']
  try:
    attachments = [item['attachments'][0]['photo']['sizes'][4]['url']]
  except:
    attachments = {}
  return {'owner_id': owner_id,
          'id': id,
          'attachments': attachments
          }

In [ ]:
# берём посты групп за 2022 год
for group in list_groups:
  # постоянный сдвиг
  offset = 0
  # это время поста, оно должно быть больше чем 01.01.2022
  time_post = 2000000000
  print(group['name'])
  group['posts'] = []
  while (time_post>time_2022):
    posts = api.wall.get(domain=group['name'], offset=offset, count=100)['items']
    posts_list = list(map(lambda item: set_posts(item), posts))
    # в словаре добавляем посты к конкретной группе (к которой они принадлежат)
    group['posts'].append(posts_list)
    # сдвигаем
    offset += 100
    # смотрим какая дата у посленего взятого нами поста
    time_post = posts[-1]['date']
    # спим
    time.sleep(0.33)

overhear
club76791337
club148881515
pmmgo
intimate_fact
podslushanoperm
club160966228
overhear_women_secrets
casesmen
overstory163
overhear_in_football
news_show_businesss
overheard_in_yaroslavl
ispovedano
podslushano_y_mam
podslushano_stroitelstvo
podslyshanobiss
yarchat
overhear_hogwarts
overhear_cosmetics
sch.story
fsb_overhear
showvk
podslushanoklubromantiki
pmmjournal
club155622384
overhear_wot
overhear_police
ufa_overhear
sbpears
secret_32
normilinet
yumorxxl
podslushanovrostove
club164570113
podslushanokrd
publictiktokp5
girlrumors
irknim
orb_overhear
secretsem
overhearmedic
podslushanomama2016
secrets_of_programmist
podsluhano_vo_vladimire
overhearkazan
krsk_overhear
metrooverhear
krskoverhear
podslushanovptz


In [ ]:
comments = api.wall.getComments(owner_id=-144260965,
                                      offset=offset,
                                      post_id=4384376,
                                      count=100,
                                      )
comments

{'can_post': True,
 'count': 59,
 'current_level_count': 31,
 'items': [],
 'show_reply_button': True}

In [ ]:
# взятие комментариев с каждого поста в каждом сообществе
# проходимся по всем группам
for group in list_groups:
  # печатаем имя группы, чтоб знать на каком мы этапе
  print(group['name'])
  group['count_comments'] = []
  # проходимся по всем постам
  for post in group['posts'][0]:
    post['comments'] = []
    count_comments = 0
    offset = 0
    flag = 1
    # берём 100 комментариев с поста
    while(flag != 0):
      comments = api.wall.getComments(owner_id=post['owner_id'],
                                      offset=offset,
                                      post_id=post['id'],
                                      count=100,
                                      )
      time.sleep(0.33)

      count_comments += comments['count']
      for comment in comments['items']:
        try:
          # берём из комментариев только ссылки на картинки и сохраняем их списком в наш словарь
          images_from_comment = comment['attachments'][0]['photo']['sizes'][4]['url']
          post['comments'].append(images_from_comment)
        except:
          pass
      if comments['count'] > (offset+100):
        offset += 100
      else:
        flag = 0
    group['count_comments'].append(count_comments)
    

overhear
club76791337
club148881515
pmmgo
intimate_fact
podslushanoperm
club160966228
overhear_women_secrets
casesmen
overstory163
overhear_in_football
news_show_businesss
overheard_in_yaroslavl
ispovedano
podslushano_y_mam
podslushano_stroitelstvo
podslyshanobiss
yarchat
overhear_hogwarts
overhear_cosmetics
sch.story
fsb_overhear
showvk
podslushanoklubromantiki
pmmjournal
club155622384
overhear_wot
overhear_police
ufa_overhear
sbpears
secret_32
normilinet
yumorxxl
podslushanovrostove
club164570113
podslushanokrd
publictiktokp5
girlrumors
irknim
orb_overhear
secretsem
overhearmedic
podslushanomama2016
secrets_of_programmist
podsluhano_vo_vladimire
overhearkazan
krsk_overhear
metrooverhear
krskoverhear
podslushanovptz


In [ ]:
# сохраняем на google drive
for group in list_groups:
  url_list = []
  name = group['name']

  try:
    os.mkdir(f"/content/drive/MyDrive/vk_api/{name}/")
  except:
    print(f'папка {name} уже создана')

  for item in group['posts'][0]:
    url_list += item['attachments']
    url_list += item['comments']
    
  for num_page, url_img in enumerate(url_list):
    urllib.request.urlretrieve(url_img, f"/content/drive/MyDrive/vk_api/{name}/{num_page}.jpg")


In [ ]:
info_of_group = {}
for group in list_groups:
  mean_of_comments = round(sum(group['count_comments'])/len(group['count_comments']),3)
  print(group['id'], group['name'], group['volume_users'], mean_of_comments)
  info_of_group[group['name']] = {'id': group['id'], 
                                  # 'name': group['name'],
                                  'members': group['volume_users'], 
                                  'comments': mean_of_comments
                                  }

34215577 overhear 4031170 13.5
76791337 club76791337 731473 0.02
148881515 club148881515 699584 0.02
144260965 pmmgo 674921 920.46
35179056 intimate_fact 496836 7.33
60397113 podslushanoperm 479792 175.2
160966228 club160966228 476585 0.02
84808204 overhear_women_secrets 392308 7.85
165972901 casesmen 377635 1.22
42949290 overstory163 365591 217.04
80142104 overhear_in_football 344005 7.58
159631764 news_show_businesss 343534 189.0
77004850 overheard_in_yaroslavl 336253 2.82
53272975 ispovedano 308767 5.11
64209172 podslushano_y_mam 293972 24.99
44150887 podslushano_stroitelstvo 282219 3.63
104144232 podslyshanobiss 280865 29.35
64058969 yarchat 274462 170.47
62418732 overhear_hogwarts 271690 3.24
87822007 overhear_cosmetics 270232 29.36
145515240 sch.story 266475 16.69
97336576 fsb_overhear 262217 426.98
165631795 showvk 260668 6.25
94672807 podslushanoklubromantiki 260251 193.07
48540538 pmmjournal 256865 139.59
155622384 club155622384 256322 0.02
61343704 overhear_wot 252652 88.05
2

In [ ]:
# list_groups[40]['count'] = 194383
info_of_group

{'casesmen': {'comments': 1.22, 'id': 165972901, 'members': 377635},
 'club148881515': {'comments': 0.02, 'id': 148881515, 'members': 699584},
 'club155622384': {'comments': 0.02, 'id': 155622384, 'members': 256322},
 'club160966228': {'comments': 0.02, 'id': 160966228, 'members': 476585},
 'club164570113': {'comments': 0.02, 'id': 164570113, 'members': 213751},
 'club76791337': {'comments': 0.02, 'id': 76791337, 'members': 731473},
 'fsb_overhear': {'comments': 426.98, 'id': 97336576, 'members': 262217},
 'girlrumors': {'comments': 310.64, 'id': 153312271, 'members': 207246},
 'intimate_fact': {'comments': 7.33, 'id': 35179056, 'members': 496836},
 'irknim': {'comments': 84.55, 'id': 47220519, 'members': 206945},
 'ispovedano': {'comments': 5.11, 'id': 53272975, 'members': 308767},
 'krsk_overhear': {'comments': 185.77, 'id': 59804801, 'members': 186316},
 'krskoverhear': {'comments': 16.35, 'id': 59717127, 'members': 176356},
 'metrooverhear': {'comments': 4.58, 'id': 79010330, 'memb